# Apriori
This repository contains a basic implementation of the (randomized) Apriori algorithm for mining frequent item sets using the market-basket model. We compare the effectiveness and efficience of the two algorithms by running them on real data sets.

In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import requests
from src.apriori import apriori

## 1. Load Data
First, let's load some testing data sets. The input format of the data is one line per basket, and each line contains the IDs of the items in the basket separated by space. We can assume that the IDs are 0,1,2,... More specifically, let's consider the following two data sets:
- *Retail*: A retail market basket dataset supplied by a anonymous Belgian retail supermarket store (small data set).
- *WebDocs*: A huge real-life transactional dataset built from a spidered collection of web html documents (huge data set).

We download these data sets from the web first.

In [3]:
execute_download: bool = False # change this line for download
if execute_download:
    # retails data set
    with open('./data/retail.dat.gz', 'wb') as outfile:
        content = requests.get('http://fimi.uantwerpen.be/data/retail.dat.gz', stream=True).content
        outfile.write(content)

    # webdocs data set
    with open('./data/webdocs.dat.gz', 'wb') as outfile:
        content = requests.get('http://fimi.uantwerpen.be/data/webdocs.dat.gz', stream=True).content
        outfile.write(content)

Go ahead and unzip these files using the preferred method on your OS such that you have the raw `.dat` files.

The big webdocs dataset shows why we often prefer approximate results: Here, the standard algorithm may be very slow, whereas a good implementation (without any smart techniques) of the randomized algorithm should be able to produce reasonably good results much faster.

## 2. Implementation Details
We implement two versions of the Apriori algorithm.

### 2.1. Standard Apriori
First, let's consider the standard version. The algorithm makes multiple passes over the data. 
- In the first pass, it counts the frequency of all items; then, it discards those whose frequency is below the threshold $t$.
- The set of frequent items is then used to generate the candidate pairs. In a second pass, the actual frequent pairs are computed.
- The sets of frequent items and frequent pairs is used to generate the candidate triples. 

Generally speaking, after $k$ passes over the data, the algorithm maintains the set of frequent $j$-itemsets (of size $j$), for every $j\in\{1,2,...,k\}$, and uses the information on frequent items and frequent $k$-itemsets to generate the candidate $k + 1$-itemsets to be verified in the next pass. The algorithm stops when no frequent k-itemsets are found.

The algorithm takes in input the path of the dataset on disk, and an integer threshold $t$. It returns a dictionary that has frequent itemsets as keys with their frequency as values.

### 2.2. Randomized Apriori
Apriori itself can take very long. For this reason, we also consider a randomized sample-based version of the algorithm that works as follows.

1. The algorithm takes in input the path of the dataset on disk, an integer threshold $t$, a sampling probability $p$, and a boolean $f$ that will allow for an extra pass over the data to remove false positives.
2. The first step of the algorithm is that of creating the sample: read the dataset once, sample each basket with probability $p$ and append it into a list $s$ to be maintained in main memory.
3. Run Apriori on $s$. Differently from the previous implementation, here we will proceed in passes over $s$ and will not read from disk. With a good implementation it might be possible to re-use most of the code written above.
4. If $f$ is true, perform an extra pass over the data on disk to remove false positives.

In [42]:
output = apriori(file_path='./data/retail.dat', t=700)

Pass 1 finished.
Pass 2 finished.
Pass 3 finished.
Pass 4 finished.


In [40]:
len(output)

249